In [30]:
import pandas as pd
import plotly.graph_objects as go
import h5py
import config
from data_processing import load_LOBs_data_by_date

In [31]:
def animate_stock_prices(lob_df, price_type='Bid', width_of_time=100):
    # Load LOB data
    if lob_df is None:
        print("Failed to load data.")
        return
    print(lob_df)
    
    # Filter by price type (Bid or Ask)
    df_prices = lob_df[lob_df['Type'] == price_type]
    
    # Ensure the dataframe is sorted by Timestamp
    df_prices.sort_values(by='timestamp', inplace=True)
    
    # Create the initial figure and layout
    fig = go.Figure(
        layout=go.Layout(width=1000, height=400,
                         title=f"{price_type} prices over time",
                         xaxis_title="Timestamp",
                         yaxis_title="Price levels",
                         template='plotly_dark',
                         hovermode="closest",
                         updatemenus=[dict(type="buttons",
                                           showactive=True,
                                           x=0.01,
                                           xanchor="left",
                                           y=1.15,
                                           yanchor="top",
                                           font={"color": 'blue'},
                                           buttons=[dict(label="Play",
                                                         method="animate",
                                                         args=[None])])]))
    
    # Set the initial frame of the animation
    fig.add_trace(go.Scatter(x=df_prices['timestamp'][:width_of_time], 
                             y=df_prices['price'][:width_of_time], 
                             mode="lines", 
                             line=dict(width=2, color="blue")))
    
    # Generate frames for the animation
    frames = [go.Frame(data=[go.Scatter(
        x=df_prices['timestamp'][k:k+width_of_time], 
        y=df_prices['price'][k:k+width_of_time], 
        mode="lines",
        line=dict(color="blue", width=2))]) 
              for k in range(1, len(df_prices) - width_of_time + 1)]
    
    fig.frames = frames
    
    # Show the figure
    fig.show()


lob_df = load_LOBs_data_by_date(config.LOBs_hdf5_path, '2025-05-13')

# Example usage
animate_stock_prices(lob_df.head(400), price_type='Bid', width_of_time=100)


     timestamp  price  quantity Type
0        1.612    1.0       1.0  Bid
1        2.170    1.0       1.0  Bid
2        2.449    2.0       1.0  Bid
3        2.945    3.0       1.0  Bid
4        3.193   16.0       5.0  Bid
..         ...    ...       ...  ...
395     13.206  175.0       2.0  Bid
396     13.206  171.0       1.0  Bid
397     13.206  170.0       9.0  Bid
398     13.206  130.0       1.0  Bid
399     13.206  117.0       1.0  Bid

[400 rows x 4 columns]


In [32]:
bids=lob_df[lob_df['Type']=='Bid']
asks=lob_df[lob_df['Type'] == 'Ask']
bids=bids.head(400)
asks=asks.head(400)
print(bids)
print(asks)

     timestamp  price  quantity Type
0        1.612    1.0       1.0  Bid
1        2.170    1.0       1.0  Bid
2        2.449    2.0       1.0  Bid
3        2.945    3.0       1.0  Bid
4        3.193   16.0       5.0  Bid
..         ...    ...       ...  ...
395     13.206  175.0       2.0  Bid
396     13.206  171.0       1.0  Bid
397     13.206  170.0       9.0  Bid
398     13.206  130.0       1.0  Bid
399     13.206  117.0       1.0  Bid

[400 rows x 4 columns]
         timestamp  price  quantity Type
2449876   2.170000  181.0       2.0  Ask
2449877   2.449000  181.0       2.0  Ask
2449878   2.945000  181.0       2.0  Ask
2449879   3.193000  181.0       2.0  Ask
2449880   3.689000  181.0       2.0  Ask
...            ...    ...       ...  ...
2450271  16.150999  486.0       2.0  Ask
2450272  16.150999  623.0       1.0  Ask
2450273  16.181999  178.0       3.0  Ask
2450274  16.181999  183.0       5.0  Ask
2450275  16.181999  400.0       3.0  Ask

[400 rows x 4 columns]


In [34]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from bisect import insort_left, bisect_left

# Assuming lob_df is your pre-loaded DataFrame containing the LOB data.


lob_df=pd.concat([lob_df.head(400),lob_df.tail(400)])
print(lob_df)




# Initialize the lists to hold the top bids and asks
top_bids = []
top_asks = []

def update_top_orders(top_orders, new_order, n, order_type):
    # Find the position where the new order should be inserted
    price = new_order['price'] * (-1 if order_type == 'Bid' else 1)  # Negate price for bids to sort in reverse
    pos = bisect_left([o['price'] for o in top_orders], price)
    
    # Insert the new order into the sorted list if appropriate
    if len(top_orders) < n or price > top_orders[-1]['price']:
        insort_left(top_orders, new_order)
        
        # If the list is too long, remove the lowest (for bids) or highest (for asks) order
        if len(top_orders) > n:
            top_orders.pop()


# Initialize figure with subplots
fig = make_subplots(rows=1, cols=1)

# Set the layout for the figure
fig.update_layout(
    width=1000,
    height=600,
    title="Limit Order Book",
    xaxis_title="Quantity",
    # y-axis labels are not needed, we'll disable them:
    yaxis=dict(showticklabels=False),
    template='plotly_dark',
    hovermode="closest",
    showlegend=True,
)

# Define the frames for animation based on timestamps
frames = []
# Loop over each timestamp
for timestamp in sorted(lob_df['timestamp'].unique()):
    # Get all new orders at the current timestamp
    new_orders = lob_df[lob_df['timestamp'] == timestamp]
    
    # Update top bids and asks
    for _, new_order in new_orders.iterrows():
        if new_order['Type'] == 'Bid':
            update_top_orders(top_bids, new_order, 10, 'Bid')
        else:
            update_top_orders(top_asks, new_order, 10, 'Ask')
    
    # Now top_bids and top_asks hold the top 10 orders at the current timestamp
    # You can use these lists to create your animation frames
        # Create a frame for the current timestamp
    frame = go.Frame(
        data=[
            go.Bar(y=top_bids['price'], x=-top_bids['quantity'], orientation='h', marker=dict(color='green'), name='Bids'),
            go.Bar(y=top_asks['price'], x=top_asks['quantity'], orientation='h', marker=dict(color='red'), name='Asks')
        ],
        name=str(timestamp)
    )
    frames.append(frame)

# Add frames to the figure
fig.frames = frames

# Add static traces for initial frame
fig.add_trace(go.Bar(y=frames[0].data[0].y, x=frames[0].data[0].x, orientation='h', marker=dict(color='green'), name='Bids'), row=1, col=1)
fig.add_trace(go.Bar(y=frames[0].data[1].y, x=frames[0].data[1].x, orientation='h', marker=dict(color='red'), name='Asks'), row=1, col=1)

# Add play button for the animation
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            showactive=False,
            y=1,
            x=0.1,
            xanchor='right',
            yanchor='top',
            buttons=[dict(label='Play',
                          method='animate',
                          args=[None, dict(frame=dict(duration=500, redraw=True), fromcurrent=True, mode='immediate')])]
        )
    ]
)

# Show the figure
fig.show()


            timestamp  price  quantity Type
0            1.612000    1.0       1.0  Bid
1            2.170000    1.0       1.0  Bid
2            2.449000    2.0       1.0  Bid
3            2.945000    3.0       1.0  Bid
4            3.193000   16.0       5.0  Bid
...               ...    ...       ...  ...
4086456  30599.945312  182.0       5.0  Ask
4086457  30599.945312  183.0       4.0  Ask
4086458  30599.945312  242.0       1.0  Ask
4086459  30599.945312  466.0       4.0  Ask
4086460  30599.945312  703.0       1.0  Ask

[800 rows x 4 columns]


TypeError: list indices must be integers or slices, not str